In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import time
import datetime
from sklearn import ensemble
from sklearn import metrics
from sklearn import linear_model
%matplotlib inline

# Подход 1: Градиентный бустинг "в лоб"

In [2]:
features = pd.read_csv('features.csv', index_col='match_id')
features_test = pd.read_csv('features_test.csv', index_col='match_id')

In [18]:
# Drop features with dependencies on result of game
x_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
                         'barracks_status_radiant', 'barracks_status_dire'], axis = 1)
x_test = features_test
x_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


In [19]:
empties = (x_train.count() != x_train.shape[0])
missing = list(x_train.loc[:,empties].columns.values)
print ('Empty observations in train dataset:')
print (missing)

Empty observations in train dataset:
['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']


In [20]:
# Fill missing values with 0
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [21]:
# Target
y = features[['radiant_win']]
y = y.fillna(0)

In [22]:
# Cross Validation
kf = KFold(x_train.shape[0], n_folds = 5, random_state = 42, shuffle = True)

n_estimators = np.arange(10, 70, 10)

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators = n)
    scores = cross_val_score (clf, x_train, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('Number of estimators: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

Number of estimators: 10, mean score: 0.664850693272, total time: 0:01:10.832713
Number of estimators: 20, mean score: 0.682461881041, total time: 0:02:18.391240
Number of estimators: 30, mean score: 0.690006467861, total time: 0:03:23.684407
Number of estimators: 40, mean score: 0.694038721334, total time: 0:04:34.859514
Number of estimators: 50, mean score: 0.697494359888, total time: 0:06:00.854270
Number of estimators: 60, mean score: 0.700119313555, total time: 0:05:56.051582


# Отчет

### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках? 
- first_blood_time
- first_blood_team
- first_blood_player1
- first_blood_player2
- radiant_bottle_time
- radiant_courier_time
- radiant_flying_courier_time
- radiant_first_ward_time
- dire_bottle_time
- dire_courier_time
- dire_flying_courier_time
- dire_first_ward_time
 
 
Так как мы имеем дело с данными на 5-ю минуту игры, то причина пропусков в данных заключается в том, что некоторые события могли не наступить в первые 5 минут игры. 
 
Например, пропуск в признаке `first_blood_player2` означает, что второй игрок не получил "первой крови" за первые 5 минут игры. Пропуск в признаке `radiant_courier_time` означает, что за первые 5 игровых минут команда не приобрела предмет "courier".
 
### 2. Как называется столбец, содержащий целевую переменную?
radiant_win

### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось? 

Время: 0:02:56.138297

Качество: 0.690006467861

### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

При увеличении деревьев можно заметить рост качества, но довольно медленный (0.69 при 30 деревьях vs 0.7 при 60 деревьях).

Один из способов ускорения — уменьшение размерности выборки. 


# Подход 2: Логистическая регрессия

In [23]:
# Some preprocessing
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [24]:
# Cross Validation
kf = KFold(x_train.shape[0], n_folds = 5, random_state = 42, shuffle = True)

n_estimators = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty = 'l2', C = n)
    scores = cross_val_score(clf, x_train_scaled, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('C: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

C: 1e-06, mean score: 0.68818505344, total time: 0:00:02.876036
C: 1e-05, mean score: 0.695161681227, total time: 0:00:03.473944
C: 0.0001, mean score: 0.711357748203, total time: 0:00:05.478690
C: 0.001, mean score: 0.71636353888, total time: 0:00:09.645762
C: 0.01, mean score: 0.716550270785, total time: 0:13:27.998685
C: 0.1, mean score: 0.716527152902, total time: 0:00:14.280656
C: 1, mean score: 0.716522605658, total time: 0:00:15.834283
C: 10, mean score: 0.716522287837, total time: 0:00:17.045146
C: 100, mean score: 0.716522295257, total time: 0:00:17.097881
C: 1000, mean score: 0.716522322797, total time: 0:00:15.779067


In [25]:
# Removing categorical variables
x_train = x_train.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)
x_train_scaled = scaler.fit_transform(x_train)

x_test = x_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)
x_test_scaled = scaler.fit_transform(x_test)

In [26]:
# Cross Validation after removing categorical variables
kf = KFold(x_train.shape[0], n_folds = 5, random_state = 42, shuffle = True)

n_estimators = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty = 'l2', C = n)
    scores = cross_val_score(clf, x_train_scaled, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('C: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

C: 1e-06, mean score: 0.688141798672, total time: 0:00:02.690650
C: 1e-05, mean score: 0.695100187107, total time: 0:00:02.937577
C: 0.0001, mean score: 0.711339145863, total time: 0:00:04.784011
C: 0.001, mean score: 0.716375799908, total time: 0:00:09.559600
C: 0.01, mean score: 0.716559386445, total time: 0:00:11.615022
C: 0.1, mean score: 0.716534239287, total time: 0:00:12.597776
C: 1, mean score: 0.71653036557, total time: 0:00:13.781546
C: 10, mean score: 0.716529971503, total time: 0:00:15.230959
C: 100, mean score: 0.716529889947, total time: 0:00:13.778177
C: 1000, mean score: 0.716529888885, total time: 0:00:15.711253


In [10]:
heroes = features[['r1_hero','r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                      'd1_hero','d2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
print('Unique heroes:', np.shape(np.unique(heroes))[0])

('Unique heroes:', 108)


In [11]:
n_heroes = 112
x_pick_train = np.zeros((x_train.shape[0], n_heroes))
for i, match_id in enumerate(x_train.index):
    for p in range(1,5):
        x_pick_train[i, features.ix[match_id, 'r%d_hero' % p] -1] = 1
        x_pick_train[i, features.ix[match_id, 'd%d_hero' % p] -1] = -1
        
x_pick_test = np.zeros((x_test.shape[0], n_heroes))
for i, match_id in enumerate(x_test.index):
    for p in range(1,5):
        x_pick_test[i, features_test.ix[match_id, 'r%d_hero' % p] -1] = 1
        x_pick_test[i, features_test.ix[match_id, 'd%d_hero' % p] -1] = -1

In [32]:
x_train_scaled_prep = pd.DataFrame(data = x_train_scaled, index = range(np.shape(x_train_scaled)[0]))
x_pick_train_prep = pd.DataFrame(data = x_pick_train, index = range(np.shape(x_pick_train)[0]))
x_train = pd.concat([x_train_scaled_prep, x_pick_train_prep], axis = 1)
print(np.shape(x_train))

x_test_scaled_prep = pd.DataFrame(data = x_test_scaled, index = range(np.shape(x_test_scaled)[0]))
x_pick_test_prep = pd.DataFrame(data = x_pick_test, index = range(np.shape(x_pick_test)[0]))
x_test = pd.concat([x_test_scaled_prep, x_pick_test_prep], axis = 1)
print(np.shape(x_test))

(97230, 203)
(17177, 203)


In [13]:
# Cross Validation after adding a pack of words
kf = KFold(x_train.shape[0], n_folds = 5, random_state = 42, shuffle = True)

n_estimators = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty = 'l2', C = n)
    scores = cross_val_score(clf, x_train, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('C: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

C: 1e-06, mean score: 0.689712653489, total time: 0:00:04.235884
C: 1e-05, mean score: 0.697812024991, total time: 0:00:04.060698
C: 0.0001, mean score: 0.720905036821, total time: 0:00:05.742972
C: 0.001, mean score: 0.738922161992, total time: 0:00:11.207817
C: 0.01, mean score: 0.74378083121, total time: 0:00:19.328975
C: 0.1, mean score: 0.743922124828, total time: 0:00:27.519533
C: 1, mean score: 0.743894337522, total time: 0:00:31.759232
C: 10, mean score: 0.743891459843, total time: 0:00:32.186258
C: 100, mean score: 0.743891137733, total time: 0:00:33.363867
C: 1000, mean score: 0.743891200206, total time: 0:00:34.600890


In [33]:
clf = LogisticRegression(penalty = 'l2', C = 0.01)
clf.fit(x_train, np.ravel(y))

predictions = clf.predict_proba(x_test)[:, 1]

In [34]:
print('Min value: ', predictions.min())
print('Max value: ', predictions.max())

('Min value: ', 0.0050398029789070278)
('Max value: ', 0.99629459142459376)


# Отчет
### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Качество: 0.717 при $(С = 0.01)$, что превышает качество градиентного бустинга чуть больше чемна 0.2. Вероятно, это связано с недообучением градиентного бустинга. Логистическая регрессия работает гораздо быстрее градиентного бустинга (~13.5 секунд vs ~3.5 минуты соответственно).
### 2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Качество незначительно увеличивается (десятитысячные доли: 0.716550 vs 0.716559). Вероятно, это связано с тем, что при большем количестве признаков модель переобучается.  

### 3. Сколько различных идентификаторов героев существует в данной игре?

108
### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?


Качество улучшилось. При $C=0.01$ было 0.716559, стало 0.74378. Улучшение качества связано с добавлением новых признаков. 
### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Минимальное значение — 0.005

Максимальное значение – 0.996
